### 04 직렬화와 역직렬화로 모델 저장 및 로드하기

- **직렬화**: 데이터 구조나 객체의 상태를 저장하거나 전송하기 위해 일련의 바이트나 문자열 형식으로 변환하는 과정
- **역직렬화**: 직렬화된 데이터를 원래의 데이터 구조나 객체로 복원하는 과정

**직렬화 가능 여부 확인하기**

- 직렬화는 직렬화가 가능한 타입만 변환 가능
- 모델의 직렬화 가능 여부 확인하는 방법

In [1]:
from dotenv import load_dotenv

from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

- 클래스(class) 에 대하여 직렬화 가능 여부를 확인

In [3]:
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


- llm 객체에 대하여 직렬화 가능 여부를 확인

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


- 체인도 직렬화 가능한지 확인

In [5]:
# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

**체인(Chain) 직렬화(dumps, dumpd)**

- 개요
  - 체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미

- 직렬화 방법
  - 객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환
  - 이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성 할 수 있도록 함

**참고**
- `dumps`: 객체를 JSON 문자열로 직렬화
- `dumpd`: 객체를 딕셔너리로 직렬화

- dumpd() 함수를 사용하여 직렬화된 체인을 확인

In [6]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [7]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumpd_chain)

dict

- dumps() 함수를 사용하여 체인을 문자열로 직렬화

In [8]:
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

In [9]:
type(dumps_chain)

str

**Pickle 파일로 직렬화하고 로드하기**

- 개요
  - Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷
- 특징
  1. **형식:**
     - Python 객체를 바이너리 형태로 직렬화하는 포맷
  2. **특징:**
     - Python 전용 (다른 언어와 호환 불가)
     - 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
     - 객체의 상태와 구조를 그대로 보존
  3. **장점:**
     - 효율적인 저장 및 전송
     - 복잡한 객체 구조 유지
     - 빠른 직렬화/역직렬화 속도

  4. **단점:**
     - 보안 위험 (신뢰할 수 없는 데이터 역직렬화 시 주의 필요)
     - 사람이 읽을 수 없는 바이너리 형식

- 주요 용도
  1. 객체 캐싱
  2. 머신러닝 모델 저장
  3. 프로그램 상태 저장 및 복원

**사용법**
- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드

- Pickle 파일로 저장

In [10]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

- JSON 형식으로도 저장 가능

In [20]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

- 전에 저장한 `pickle` 형식의 파일을 로드

In [14]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

- 로드한 json 파일을 `load` 메서드를 사용하여 로드

In [15]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

/var/folders/5h/jssvspks5fn6v_wtpcsmpp580000gn/T/ipykernel_83018/4150181846.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D7PH4zXI3KyICHSYC5WdpuJF4ExEE', 'finish_reason': 'stop', 'logprobs': None} id='run-4b8d4b1f-ed97-435c-b3eb-ec2205780572-0' usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


- OpenAI API 키를 환경 변수에서 불러와 인증 정보로 설정 가능

In [16]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D7PJNmbObgWcFTgOPcXAdunRveA9m', 'finish_reason': 'stop', 'logprobs': None}, id='run-951c9f6a-68b8-46ef-8c27-741c3abdad2e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- JSON 파일을 읽기모드로 열고, JSON 데이터를 파이썬 객체로 변환

In [19]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D7PM0ZKN9tUg2HJGOSeS0j1zomFfv', 'finish_reason': 'stop', 'logprobs': None}, id='run-540d2ef3-85c9-470e-942d-96c2e121aa85-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})